PART 2: SQLite
---

In [1]:
import sqlite3
from sqlite3 import Error

try:
    conn = sqlite3.connect('sqlite_db_pythonsqlite.db')
    print(sqlite3.version)
except Error as e:
    print(e)

cur = conn.cursor()

def print_query_result(sql) :
    cur.execute(sql)

    rows = cur.fetchall()
    for row in rows:
        print(row)

2.6.0


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [2]:
print_query_result("""
            
WITH booking_plus AS (
SELECT
    b.bookid,
    b.facid,
    b.memid,
    CASE WHEN m.surname = 'GUEST' THEN f.guestcost * b.slots
        ELSE f.membercost * b.slots END AS booking_revenue
    FROM Bookings AS b
    LEFT JOIN Facilities AS f ON b.facid = f.facid
    LEFT JOIN Members AS m ON b.memid = m.memid
)

SELECT 
    f.name,
    SUM(booking_revenue) AS revenue
FROM booking_plus AS b
LEFT JOIN Facilities AS f ON b.facid = f.facid
GROUP BY f.name
HAVING SUM(booking_revenue) < 1000
ORDER BY revenue
            
""")

('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)


Q11: Produce a report of members and who recommended them in alphabetic surname, firstname order

In [3]:
# I'm going to assume that, if the member was not recommended, we also want to see that information.
# If we wanted to filter out the records with no recommender, we'd just add:

#       WHERE recommendedby IS NOT NULL

# This expression could be added to either query (though it would need an AND if it's in the CTE).

print_query_result("""
                   
WITH members_only AS (
    SELECT
        firstname,
        surname,
        recommendedby,
        memid
    FROM members
    WHERE firstname != 'Guest'
)

SELECT
    m1.surname,
    m1.firstname,
    m2.firstname||' '||m2.surname AS recommender
FROM members_only AS m1
LEFT JOIN members_only AS m2 ON m1.recommendedby = m2.memid
ORDER BY m1.surname, m1.firstname

""")

('Bader', 'Florence', 'Ponder Stibbons')
('Baker', 'Anne', 'Ponder Stibbons')
('Baker', 'Timothy', 'Jemima Farrell')
('Boothe', 'Tim', 'Tim Rownam')
('Butters', 'Gerald', 'Darren Smith')
('Coplin', 'Joan', 'Timothy Baker')
('Crumpet', 'Erica', 'Tracy Smith')
('Dare', 'Nancy', 'Janice Joplette')
('Farrell', 'David', None)
('Farrell', 'Jemima', None)
('GUEST', 'GUEST', None)
('Genting', 'Matthew', 'Gerald Butters')
('Hunt', 'John', 'Millicent Purview')
('Jones', 'David', 'Janice Joplette')
('Jones', 'Douglas', 'David Jones')
('Joplette', 'Janice', 'Darren Smith')
('Mackenzie', 'Anna', 'Darren Smith')
('Owen', 'Charles', 'Darren Smith')
('Pinker', 'David', 'Jemima Farrell')
('Purview', 'Millicent', 'Tracy Smith')
('Rownam', 'Tim', None)
('Rumney', 'Henrietta', 'Matthew Genting')
('Sarwin', 'Ramnaresh', 'Florence Bader')
('Smith', 'Darren', None)
('Smith', 'Darren', None)
('Smith', 'Jack', 'Darren Smith')
('Smith', 'Tracy', None)
('Stibbons', 'Ponder', 'Burton Tracy')
('Tracy', 'Burton', N

Q12: Find the facilities with their usage by member, but not guests

In [4]:
print_query_result("""
            
WITH member_bookings AS (
    SELECT
        facid,
        memid,
        slots
    FROM Bookings
    WHERE memid != 0
)

SELECT
    f.name AS facility,
    m.firstname||' '||m.surname AS member,
    SUM(b.slots) AS member_usage
FROM member_bookings AS b
LEFT JOIN Facilities AS f ON b.facid = f.facid
LEFT JOIN Members AS m ON b.memid = m.memid
GROUP BY facility, member
            
""")

('Badminton Court', 'Anna Mackenzie', 96)
('Badminton Court', 'Anne Baker', 30)
('Badminton Court', 'Burton Tracy', 6)
('Badminton Court', 'Charles Owen', 18)
('Badminton Court', 'Darren Smith', 432)
('Badminton Court', 'David Jones', 24)
('Badminton Court', 'David Pinker', 21)
('Badminton Court', 'Douglas Jones', 6)
('Badminton Court', 'Erica Crumpet', 6)
('Badminton Court', 'Florence Bader', 27)
('Badminton Court', 'Gerald Butters', 63)
('Badminton Court', 'Henry Worthington-Smyth', 15)
('Badminton Court', 'Hyacinth Tupperware', 3)
('Badminton Court', 'Jack Smith', 36)
('Badminton Court', 'Jemima Farrell', 21)
('Badminton Court', 'John Hunt', 6)
('Badminton Court', 'Millicent Purview', 6)
('Badminton Court', 'Nancy Dare', 30)
('Badminton Court', 'Ponder Stibbons', 48)
('Badminton Court', 'Ramnaresh Sarwin', 21)
('Badminton Court', 'Tim Boothe', 36)
('Badminton Court', 'Tim Rownam', 12)
('Badminton Court', 'Timothy Baker', 21)
('Badminton Court', 'Tracy Smith', 102)
('Massage Room 1',

Q13: Find the facilities usage by month, but not guests

In [5]:
print_query_result("""
                
WITH monthly_bookings AS (
    SELECT
        facid,
        slots,
        strftime('%Y', starttime)||'-'||strftime('%m', starttime) AS month
    FROM Bookings
    WHERE memid != 0
)

SELECT
    f.name AS facility,
    b.month,
    SUM(b.slots) AS member_usage
FROM monthly_bookings AS b
LEFT JOIN Facilities AS f ON b.facid = f.facid
GROUP BY facility, month
                
""")

('Badminton Court', '2012-07', 165)
('Badminton Court', '2012-08', 414)
('Badminton Court', '2012-09', 507)
('Massage Room 1', '2012-07', 166)
('Massage Room 1', '2012-08', 316)
('Massage Room 1', '2012-09', 402)
('Massage Room 2', '2012-07', 8)
('Massage Room 2', '2012-08', 18)
('Massage Room 2', '2012-09', 28)
('Pool Table', '2012-07', 110)
('Pool Table', '2012-08', 303)
('Pool Table', '2012-09', 443)
('Snooker Table', '2012-07', 140)
('Snooker Table', '2012-08', 316)
('Snooker Table', '2012-09', 404)
('Squash Court', '2012-07', 50)
('Squash Court', '2012-08', 184)
('Squash Court', '2012-09', 184)
('Table Tennis', '2012-07', 98)
('Table Tennis', '2012-08', 296)
('Table Tennis', '2012-09', 400)
('Tennis Court 1', '2012-07', 201)
('Tennis Court 1', '2012-08', 339)
('Tennis Court 1', '2012-09', 417)
('Tennis Court 2', '2012-07', 123)
('Tennis Court 2', '2012-08', 345)
('Tennis Court 2', '2012-09', 414)
